### 1. Load the Fish Data, split into train and test set, and standardize data.

In [54]:
import pandas as pd
import numpy
fish_data = pd.read_csv('fish2.csv')
fish_data

,Species,Length1,Length2,Length3,Height,Width,Weight
0,Bream,23.2,25.4,30.0,11.5200,4.0200,242.0
1,Bream,24.0,26.3,31.2,12.4800,4.3056,290.0
2,Bream,23.9,26.5,31.1,12.3778,4.6961,340.0
3,Bream,26.3,29.0,33.5,12.7300,4.4555,363.0
4,Bream,26.5,29.0,34.0,12.4440,5.1340,430.0
...,...,...,...,...,...,...,...
154,Smelt,11.5,12.2,13.4,2.0904,1.3936,12.2
155,Smelt,11.7,12.4,13.5,2.4300,1.2690,13.4
156,Smelt,12.1,13.0,13.8,2.2770,1.2558,12.2
157,Smelt,13.2,14.3,15.2,2.8728,2.0672,19.7


In [55]:
## Create Dummy Variables and convert to torch
import torch
X = fish_data[['Length1', 'Length2', 'Length3', 'Height', 'Width', 'Species']]
X = torch.tensor(pd.get_dummies(X, dtype=float).to_numpy(), dtype=torch.float)
y = torch.tensor(fish_data[['Weight']].to_numpy(), dtype=torch.float)

In [56]:
## Split into train and test set
generator = torch.Generator().manual_seed(42)
idx = torch.randperm(X.shape[0], generator=generator)
X_train, X_test = X[idx[:127],:], X[idx[127:],:]
y_train, y_test = y[idx[:127],:], y[idx[127:],:]

In [57]:
## Standardize Data
# Lambda Function
standardize = lambda z, z_mean, z_std: (z - z_mean) / (z_std + 1e-5) 

# Standardize X
x_mean, x_std = X_train[:,:5].mean(0, keepdims=True), X_train[:,:5].std(0, keepdims=True)
X_train[:,:5], X_test[:,:5] = standardize(X_train[:,:5], x_mean, x_std), standardize(X_test[:,:5], x_mean, x_std)
# Standardize y
y_mean, y_std = y_train.mean(0, keepdims=True), y_train.std(0, keepdims=True)
y_train = standardize(y_train, y_mean, y_std)

In [59]:
print(y_test.type)

<built-in method type of Tensor object at 0x0000021D92EADA30>


### 2 Create k Nearest Neighbor Regressor class

In [ ]:
class kNNRegressor:
    # k: Number of k Nearest Neighbors
    # p: L_p distance function
    # Note: output label is simple average of k Nearest Neighbors
    def __init__(self, k, p=2.0):
        self.k = k
        self.p = p
        self.train_x = None
        self.train_y = None
    
    def fit(self, X, y):
        """
        Parameters
        ----------
        X : Array of shape [n_samples, n_features]
        y : Array of shape [n_samples, 1]
        """  
        ...
        return self
        
    def predict(self, X, mean=None, std=None):
        """
        Parameters
        ----------
        X : Array of shape [n_samples, n_features]
        """
        # Calculate distance matrix
        dist = ...
        dist = dist.pow(self.p).mean(2).pow(1./self.p)
        
        # Get k nearest neighbots
        _, idx_arg = ...
        
        # Collect labels
        train_y = ... # First repeat train labels to match idx_arg
        predictions = ... # Next, gather train labels
        predictions = ... # Finally, calculate mean
        
        # Recompute unstandardized data
        if mean is not None and std is not None:
            predictions = ...
        return predictions

In [ ]:
k, p = 5, 2.
knn_model = kNNRegressor(k, p=p).fit(X_train, y_train)
y_pred = knn_model.predict(X_test, y_mean, y_std)
mse = ...
print(f'The MSE of the {k}-NN model with L_{p}-Distance is: {mse:10.4f}')

### 3. Create Linear Regressor class

In [52]:
class LinearRegression:
    def __init__(self):
        self.weight = None
            
    def fit(self, X, y):
        """
        Parameters
        ----------
        X : Array of shape [n_samples, n_features]
        y : Array of shape [n_samples, 1]
        """        
        
        # prepend a column of ones
        ones = numpy.ones((X.shape[0], 1))
        X = numpy.concatenate((ones, X), axis=1)

        # compute weights
        X = torch.tensor(X)
        XtX_pinv = torch.inverse(torch.matmul(X.T, X))
        X = X.float()
        Xty = torch.matmul(X.T, y)
        
        # dot: matrix multiplication
        XtX_pinv = XtX_pinv.float()
        self.weight = torch.matmul(XtX_pinv, Xty)
        return self
                
    def predict(self, X, mean=None, std=None):
        """
        Parameters
        ----------
        X : Array of shape [n_samples, n_features]
        """                     
        # prepend a column of ones
        ones = numpy.ones((X.shape[0], 1))
        X = numpy.concatenate((ones, X), axis=1)

        # compute predictions
        predictions = numpy.dot(X, self.weight)
        
        # Recompute unstandardized data
        if mean is not None and std is not None:
            std = std.numpy()
            mean = mean.numpy()
            predictions = predictions * std + mean
        return predictions

In [53]:
lr_model = LinearRegression().fit(X_train, y_train)
y_pred = lr_model.predict(X_test, y_mean, y_std)
mse = numpy.mean((y_pred - y_test.numpy()) ** 2)
print(f'The MSE of the linear regression model is: {mse:10.4f}')

The MSE of the linear regression model is: 104965.0249


### 4. Create a Linear Regressor class, but this time use PyTorch autograd function to calculate the gradient.

In [62]:
import torch.nn as nn

class LinearRegressionGradientDescent:
    def __init__(self, input_features, lr=0.1, epochs=1000):
        self.weight = nn.Parameter(torch.zeros(input_features + 1, 1))
        self.lr, self.epochs = lr, epochs
        
    def fit(self, X, y):
        """
        Parameters
        ----------
        X : Array of shape [n_samples, n_features]
        y : Array of shape [n_samples, 1]
        """        
        for _ in range(self.epochs):
            y_pred = self.forward(X)
            loss = nn.functional.mse_loss(y_pred, y)
            loss.backward()
            with torch.no_grad():
                self.weight -= self.lr * self.weight.grad
            self.weight.grad = None
        return self
    
    def forward(self, X):
        """
        Parameters
        ----------
        X : Array of shape [n_samples, n_features]
        """                     
        # prepend a column of ones
        ones = numpy.ones((X.shape[0], 1))
        X = numpy.concatenate((ones, X), axis=1)

        # compute predictions
        predictions = torch.matmul(torch.Tensor(X), self.weight)

        return predictions
    
    def predict(self, X, mean=None, std=None):
        # Execute forward pass
        predictions = self.forward(X)
        
        # Recompute unstandardized data
        if mean is not None and std is not None:
            predictions = predictions * std + mean
            
        return predictions

In [63]:
lrd_model = LinearRegressionGradientDescent(12).fit(X_train, y_train)
y_pred = lrd_model.predict(X_test, y_mean, y_std)
mse = ((y_pred - y_test)**2).mean()
print(f'The MSE of the gradient descent linear regression model is: {mse:10.4f}')

TypeError: expected Tensor as element 0 in argument 0, but got numpy.ndarray